In [10]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation

import bp
import numpy as np
# np.set_printoptions(suppress=True)

import plotly.express as px
import plotly.figure_factory as ff

### Model of the lung's health

In [54]:
U = bp.variableNode("Unblocked FEV1", 2, 6, 0.1)
C = bp.variableNode("Small airway clearance", 0.7, 1, 0.05)
FEV1 = bp.variableNode("FEV1", 0.2, 6, 0.1)

graph = BayesianNetwork([(U.name, FEV1.name), (C.name, FEV1.name)])

cpt_fev1 = TabularCPD(
    variable=FEV1.name,
    variable_card=len(FEV1.bins)-1,
    values=bp.calc_pgmpy_cpt(U, C, FEV1),
    evidence=[C.name, U.name],
    evidence_card=[len(C.bins)-1, len(U.bins)-1],
)

prior_b = TabularCPD(
    variable=C.name,
    variable_card=len(C.bins)-1,
    values=C.marginal(C),
    evidence=[],
    evidence_card=[],
)

prior_u = TabularCPD(
    variable=U.name,
    variable_card=len(U.bins)-1,
    values=U.marginal(U),
    evidence=[],
    evidence_card=[],
)

graph.add_cpds(cpt_fev1, prior_b, prior_u)

graph.check_model()

inference = BeliefPropagation(graph)


calculating cpt of shape 58 x 40 x 7 (C x (A x B)) 


NameError: name 'C_range' is not defined

### Interactive inference

In [ ]:
inference = BeliefPropagation(graph)
fev1=0.2
res = inference.query(variables=[U.name], evidence={FEV1.name: fev1})
res.values


  0%|          | 0/1 [00:00<?, ?it/s]

array([0.21428602, 0.18281378, 0.15414052, 0.12790869, 0.10381917,
       0.08161978, 0.06109644, 0.04206622, 0.02437187, 0.0078775 ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [ ]:
fig = ff.create_distplot()

In [18]:
fig = px.bar(y=res.values, x=U.bins[:-1])
fig.show()

In [22]:
res = inference.query(variables=[C.name], evidence={FEV1.name: 0})
res.values

AttributeError: module 'bp' has no attribute 'query'

In [ ]:
C.bins

1.0

In [55]:
from dash import Dash, dcc, html, Input, Output

app = Dash(__name__)

app.layout = html.Div([
    html.H4('Interactive normal distribution'),
    dcc.Graph(id="graph"),
    html.P("FEV1:"),
    dcc.Slider(id="fev1", min=FEV1.bins[0], max=FEV1.bins[-1], value=0.2, 
               marks={0: '0.2', (len(C.bins)-1): '6'}),
])


@app.callback(
    Output("graph", "figure"), 
    Input("fev1", "value"))
def display_color(fev1):
    print("get fev1", fev1)
  
    # Find in C.bins the closest value to fev1
    fev1_idx = np.argmin(np.abs(FEV1.bins - fev1))-1
    print(fev1_idx)
    res = inference.query(variables=[U.name], evidence={FEV1.name: fev1_idx})
    fig = px.bar(y=res.values, x=U.bins[:-1])
    return fig

app.run_server(debug=True, port=8049, use_reloader=False)

Dash is running on http://127.0.0.1:8049/

 * Serving Flask app '__main__'
 * Debug mode: on
get fev1 0.2
-1


/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:540: UserWarning:

Found unknown state name. Trying to switch to using all state names as state numbers



  0%|          | 0/1 [00:00<?, ?it/s]

get fev1 0.2
-1


/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:540: UserWarning:

Found unknown state name. Trying to switch to using all state names as state numbers



  0%|          | 0/1 [00:00<?, ?it/s]

get fev1 3
27


  0%|          | 0/1 [00:00<?, ?it/s]

0.1-0.05